In [1]:
#Import all the necessary libraries
import sys
import os
import requests
import time
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import pandas as pd
import pickle
import datetime as dt
import dateutil.relativedelta as ddelta

In [2]:
#set up parameters for the code
start_date = dt.date(2013,1,1) #start of the time range for weather scraping
end_date = dt.date(2018,12,31)  #end of the time range for weather scraping
weather_dates = []              #list of year-month strings to input into weather website to scrape
my_data = []                    #final output of all scraped daily weather data, to be pickled

In [3]:
#Complete Selenium setup
chromedriver = "/Applications/chromedriver" # path to the chromedriver executable
os.environ["webdriver.chrome.driver"] = chromedriver

In [4]:
#generate the list of months to be scraped
curr_date = start_date

while curr_date < end_date:
    weather_dates.append(str(curr_date.year)+'-'+str(curr_date.month))
    curr_date = curr_date + ddelta.relativedelta(months=1)

In [5]:
#start weather scraping - open the site and set station to Seattle Area
driver = webdriver.Chrome(chromedriver)
driver.get("https://nowdata.rcc-acis.org/sew/")
time.sleep(1)

#Select the Seattle Area option under "Location" menu
element = driver.find_element_by_name('station')
all_options = element.find_elements_by_tag_name("option")
all_options[2].click()

#although "daily data" is the default option, add code to select it just to be safe
product_choice = driver.find_element_by_name('product_select')
product_choice.click()

In [6]:
#scrape through the data for each month

for scrape_month in weather_dates:
    
    #Select the right year and month
    date_field = driver.find_element_by_id("tDatepicker")
    date_field.click()
    date_field.clear()
    date_field.send_keys(scrape_month)
    
    #hit Go to load the data!
    go_button = driver.find_element_by_id("go")
    go_button.send_keys(Keys.ENTER)
    time.sleep(1)
    
    #read in the resulting table
    my_results = driver.find_element_by_id("results_area")
    my_table = my_results.find_element_by_tag_name("tbody")
    my_table_rows = my_table.find_elements_by_tag_name("tr")
    
    for row in my_table_rows:
        new_row = []
        my_table_cols = row.find_elements_by_tag_name("td")
        
        for col in my_table_cols:
            new_row.append(col.text)
        my_data.append(new_row)
        
    #exit out of this view to load the next date
    close_button = my_results.find_element_by_xpath('/html/body/div[4]/div[1]/button[3]').click()
    time.sleep(1)

In [7]:
#now pickle that thing in case of apocalypse!
with open('weather2.pickle', 'wb') as to_write:
    pickle.dump(my_data, to_write)